In [3]:
import dataset
from args import *

dataset.make_dataset(args)

TypeError: make_dataset() takes 0 positional arguments but 1 was given

In [3]:
import csv

indices_filename = osp.join("{}".format(DATASET_MP_RAW_DIR), "INDICES")
assert osp.exists(indices_filename), "INDICES file not exist in " + indices_filename
with open(indices_filename) as f:
    reader = csv.reader(f)
    indices = [row for row in reader][1:]  # ignore first line of header

# data = dataset.read_one_compound_info(indices[0])
# print(data.edge_index)

In [5]:
import random
import csv
import os.path as osp
import ase
from ase.io import read as ase_read
import numpy as np
import torch
from tqdm import tqdm
from torch_geometric.data import Data, Dataset
from mp_api.client import MPRester
from torch_geometric.utils import dense_to_sparse
from torch.utils.data import random_split, Subset
from typing import Sequence

data = Data()

compound_data = indices[0]
print(compound_data)

idx, mid, y = compound_data[0], compound_data[1], compound_data[2]
filename = osp.join("{}".format(DATASET_MP_RAW_DIR), "CONFIG_" + idx + ".poscar")
compound = ase_read(filename, format="vasp")
print(compound.get_cell())

# get distance matrix
distance_matrix = compound.get_all_distances(mic=True)
print(distance_matrix)
# get mask by max cutoff distance
cutoff_mask = distance_matrix > args["max_cutoff_distance"]
# suppress invalid values using max cutoff distance
distance_matrix = np.ma.array(distance_matrix, mask=cutoff_mask)
# let '--' in the masked array to 0
distance_matrix = np.nan_to_num(np.where(cutoff_mask, np.isnan(distance_matrix), distance_matrix))
# make it as a tensor
distance_matrix = torch.Tensor(distance_matrix)
print(distance_matrix)


# dense transform to sparse to get edge_index and edge_attr
sparse_distance_matrix = dense_to_sparse(distance_matrix)
data.edge_index = sparse_distance_matrix[0]
data.edge_attr = torch.Tensor(np.array([sparse_distance_matrix[1]], dtype=np.float32)).t().contiguous()

data.x = torch.Tensor(np.array([compound.get_atomic_numbers()], dtype=np.float32)).t().contiguous()
data.y = torch.Tensor(np.array([y], dtype=np.float32))

print(data)
data.edge_attr

['1', 'mp-861724', '-0.41328523750000556']
Cell([[7.60953026, 0.0, 5e-16], [-5e-16, 7.60953026, 5e-16], [0.0, 0.0, 7.60953026]])
[[0.         3.80476513 5.38075045 6.59004652 5.38075045 3.80476513
  5.38075045 3.80476513 3.29502326 3.29502326 3.29502326 3.29502326
  3.29502326 3.29502326 3.29502326 3.29502326]
 [3.80476513 0.         6.59004652 5.38075045 3.80476513 5.38075045
  3.80476513 5.38075045 3.29502326 3.29502326 3.29502326 3.29502326
  3.29502326 3.29502326 3.29502326 3.29502326]
 [5.38075045 6.59004652 0.         3.80476513 5.38075045 3.80476513
  5.38075045 3.80476513 3.29502326 3.29502326 3.29502326 3.29502326
  3.29502326 3.29502326 3.29502326 3.29502326]
 [6.59004652 5.38075045 3.80476513 0.         3.80476513 5.38075045
  3.80476513 5.38075045 3.29502326 3.29502326 3.29502326 3.29502326
  3.29502326 3.29502326 3.29502326 3.29502326]
 [5.38075045 3.80476513 5.38075045 3.80476513 0.         3.80476513
  5.38075045 6.59004652 3.29502326 3.29502326 3.29502326 3.29502326
  3

tensor([[3.8048],
        [3.8048],
        [3.8048],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.8048],
        [3.8048],
        [3.8048],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.8048],
        [3.8048],
        [3.8048],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.8048],
        [3.8048],
        [3.8048],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.8048],
        [3.8048],
        [3.8048],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3.2950],
        [3